# Chapter 16: Multi-Agent Systems
**From: Zero to AI Agent**

## Overview
In this chapter, you'll learn about:
- When and why to use multiple agents
- Agent communication patterns
- Supervisor-worker architectures
- Collaborative agent systems
- Building a research assistant team
- Managing shared state between agents
- Orchestration and coordination


In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Section 16.1: When and why to use multiple agents

In [ ]:
# From: single_agent_demo.py

# From: Zero to AI Agent, Chapter 16, Section 16.1
# File: single_agent_demo.py

"""
Demonstrates a single agent handling multiple responsibilities.
"""

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# One big prompt trying to do everything
MEGA_PROMPT = """You are a versatile assistant that can:
1. Analyze text for sentiment, key themes, and statistics
2. Summarize content in different styles
3. Generate creative variations
4. Translate between formats

For any input, determine what the user needs and provide it.
Be analytical when analyzing, creative when creating, 
concise when summarizing."""

text = """
The new product launch exceeded expectations. Sales were up 150% 
compared to our previous launch. Customer feedback has been 
overwhelmingly positive, with 92% satisfaction ratings. However, 
we did face some supply chain challenges that delayed shipments 
to certain regions by 2-3 weeks.
"""

# Single agent tries to do analysis AND summary
response = llm.invoke(f"""
{MEGA_PROMPT}

Please analyze this text AND provide a brief executive summary:

{text}
""")

print("=== Single Agent Response ===")
print(response.content)


In [ ]:
# From: multi_agent_demo.py

# From: Zero to AI Agent, Chapter 16, Section 16.1
# File: multi_agent_demo.py

"""
Demonstrates specialized agents as graph nodes.
"""

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class ReportState(TypedDict):
    raw_text: str
    analysis: str
    summary: str


def analyst_agent(state: ReportState) -> dict:
    """Data Analyst Agent - extracts metrics and signals."""
    prompt = """You are a data analyst. Your ONLY job is to:
    - Extract key metrics and statistics
    - Identify positive and negative signals
    - Note any risks or concerns
    Be precise. Use numbers. Format as bullet points."""
    
    response = llm.invoke(f"{prompt}\n\nAnalyze:\n{state['raw_text']}")
    return {"analysis": response.content}


def summarizer_agent(state: ReportState) -> dict:
    """Executive Summarizer Agent - creates brief summaries."""
    prompt = """You are an executive communication specialist.
    Write a 2-3 sentence summary for busy executives.
    Lead with the most important finding."""
    
    response = llm.invoke(f"{prompt}\n\nBased on:\n{state['analysis']}")
    return {"summary": response.content}


# Build the multi-agent graph
workflow = StateGraph(ReportState)

# Add our specialist agents as nodes
workflow.add_node("analyst", analyst_agent)
workflow.add_node("summarizer", summarizer_agent)

# Define the flow: analyst first, then summarizer
workflow.add_edge(START, "analyst")
workflow.add_edge("analyst", "summarizer")
workflow.add_edge("summarizer", END)

# Compile the graph
app = workflow.compile()

# Run the multi-agent system
result = app.invoke({
    "raw_text": """The new product launch exceeded expectations. Sales were up 
    150% compared to our previous launch. Customer feedback has been 
    overwhelmingly positive, with 92% satisfaction ratings. However, 
    we did face some supply chain challenges that delayed shipments 
    to certain regions by 2-3 weeks.""",
    "analysis": "",
    "summary": ""
})

print("=== Analyst Agent Output ===")
print(result["analysis"])
print("\n=== Summarizer Agent Output ===")
print(result["summary"])


---
### Section 16.1 Exercises

### Exercise 16.1.1: Multi-Agent Decision Analysis

You're designing an AI system for each scenario below. For each one, decide: single agent or multiple agents? If multiple, identify the agents you'd create.

Scenarios:
1. A chatbot that answers FAQs about a company's products
2. An AI that reviews legal contracts, checks for compliance issues, and suggests revisions
3. A customer service system that handles complaints, processes refunds, and escalates complex issues
4. A translation tool that converts English documents to Spanish
5. An AI research assistant that finds papers, summarizes them, identifies gaps, and suggests experiments

Write a brief justification for each decision.

In [ ]:
# Your code here


### Exercise 16.1.2: Agent Boundary Design

You're building an AI-powered content creation pipeline for a marketing team. The workflow is:

1. Research trending topics in the industry
2. Generate content ideas based on research
3. Write first draft of article
4. Review for factual accuracy
5. Edit for brand voice and style
6. Generate social media snippets

Design the agent architecture. For each agent you propose:
- What is its name and single responsibility?
- What tools would it need?
- What context/instructions would it have?
- What does it receive as input? What does it output?

Draw a simple diagram showing how information flows between agents.

In [ ]:
# Your code here


### Exercise 16.1.3: Specialist vs. Generalist Comparison

Create a practical comparison by implementing two approaches to this task:

*"Given a piece of code, identify bugs, suggest optimizations, and add documentation."*

1. Build a single-agent version that does all three tasks
2. Build a multi-agent version with three specialists (Bug Finder, Optimizer, Documenter)
3. Run both on the same sample code
4. Compare the quality and depth of outputs

Sample code to analyze:
```python
def calculate_average(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    average = total / len(numbers)
    return average
```

Document your observations about the differences in output quality.

In [ ]:
# Your code here


---
## Section 16.2: Agent communication patterns

In [ ]:
# From: sequential_pipeline.py

# From: Zero to AI Agent, Chapter 16, Section 16.2
# File: sequential_pipeline.py

"""
Sequential pattern: Research → Draft → Edit pipeline.
"""

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


class ArticleState(TypedDict):
    topic: str
    research: str
    draft: str
    final: str


def researcher_agent(state: ArticleState) -> dict:
    """Stage 1: Research the topic."""
    prompt = f"""Research this topic and provide 3 key facts:
    Topic: {state['topic']}
    
    Format: Three bullet points with factual information."""
    
    response = llm.invoke(prompt)
    print("📚 Researcher complete")
    return {"research": response.content}


def writer_agent(state: ArticleState) -> dict:
    """Stage 2: Write draft based on research."""
    prompt = f"""Write a short paragraph about {state['topic']}.
    
    Use these research points:
    {state['research']}
    
    Keep it to 3-4 sentences."""
    
    response = llm.invoke(prompt)
    print("✍️ Writer complete")
    return {"draft": response.content}


def editor_agent(state: ArticleState) -> dict:
    """Stage 3: Polish the draft."""
    prompt = f"""Edit this draft for clarity and impact:
    
    {state['draft']}
    
    Make it more engaging. Keep the same length."""
    
    response = llm.invoke(prompt)
    print("📝 Editor complete")
    return {"final": response.content}


# Build sequential pipeline
workflow = StateGraph(ArticleState)

workflow.add_node("researcher", researcher_agent)
workflow.add_node("writer", writer_agent)
workflow.add_node("editor", editor_agent)

# Sequential flow: one after another
workflow.add_edge(START, "researcher")
workflow.add_edge("researcher", "writer")
workflow.add_edge("writer", "editor")
workflow.add_edge("editor", END)

app = workflow.compile()

# Run it
result = app.invoke({
    "topic": "The benefits of morning exercise",
    "research": "",
    "draft": "",
    "final": ""
})

print("\n" + "=" * 50)
print("FINAL ARTICLE:")
print("=" * 50)
print(result["final"])


In [ ]:
# From: broadcast_pattern.py

# From: Zero to AI Agent, Chapter 16, Section 16.2
# File: broadcast_pattern.py

"""
Broadcast pattern: Multiple reviewers analyze the same code.
"""

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class ReviewState(TypedDict):
    code: str
    security_review: str
    performance_review: str
    style_review: str
    summary: str


def security_reviewer(state: ReviewState) -> dict:
    """Reviews code for security issues."""
    prompt = f"""As a security expert, review this code for vulnerabilities:
    
    {state['code']}
    
    List any security concerns (or say 'No issues found')."""
    
    response = llm.invoke(prompt)
    print("🔒 Security review complete")
    return {"security_review": response.content}


def performance_reviewer(state: ReviewState) -> dict:
    """Reviews code for performance issues."""
    prompt = f"""As a performance expert, review this code for efficiency:
    
    {state['code']}
    
    List any performance concerns (or say 'No issues found')."""
    
    response = llm.invoke(prompt)
    print("⚡ Performance review complete")
    return {"performance_review": response.content}


def style_reviewer(state: ReviewState) -> dict:
    """Reviews code for style and readability."""
    prompt = f"""As a code quality expert, review this code for style:
    
    {state['code']}
    
    List any style/readability concerns (or say 'No issues found')."""
    
    response = llm.invoke(prompt)
    print("🎨 Style review complete")
    return {"style_review": response.content}


def aggregator(state: ReviewState) -> dict:
    """Combines all reviews into a summary."""
    prompt = f"""Summarize these code reviews into a brief action list:
    
    Security: {state['security_review']}
    Performance: {state['performance_review']}  
    Style: {state['style_review']}
    
    Prioritize the top 3 issues to fix."""
    
    response = llm.invoke(prompt)
    return {"summary": response.content}


workflow = StateGraph(ReviewState)

workflow.add_node("security", security_reviewer)
workflow.add_node("performance", performance_reviewer)
workflow.add_node("style", style_reviewer)
workflow.add_node("aggregator", aggregator)

# Fan-out: all three reviewers start from START
workflow.add_edge(START, "security")
workflow.add_edge(START, "performance")
workflow.add_edge(START, "style")

# Fan-in: all feed into aggregator
workflow.add_edge("security", "aggregator")
workflow.add_edge("performance", "aggregator")
workflow.add_edge("style", "aggregator")

workflow.add_edge("aggregator", END)

app = workflow.compile()

# Test it with some code
sample_code = """
def get_user(user_id):
    query = f"SELECT * FROM users WHERE id = {user_id}"
    result = db.execute(query)
    users = []
    for row in result:
        users.append(row)
    return users[0] if users else None
"""

result = app.invoke({
    "code": sample_code,
    "security_review": "",
    "performance_review": "",
    "style_review": "",
    "summary": ""
})

print("\n" + "=" * 50)
print("COMBINED REVIEW SUMMARY:")
print("=" * 50)
print(result["summary"])


In [ ]:
# From: supervisor_pattern.py

# From: Zero to AI Agent, Chapter 16, Section 16.2
# File: supervisor_pattern.py

"""
Supervisor pattern: Routes requests to appropriate specialists.
"""

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class CustomerState(TypedDict):
    request: str
    category: str
    response: str


def supervisor(state: CustomerState) -> dict:
    """Analyzes request and determines routing."""
    prompt = f"""Categorize this customer request into exactly one category:
    - billing (payment, charges, invoices, refunds)
    - technical (bugs, errors, how-to, features)
    - general (other questions, feedback, complaints)
    
    Request: {state['request']}
    
    Reply with just the category name."""
    
    response = llm.invoke(prompt)
    category = response.content.strip().lower()
    
    # Normalize to valid categories
    if "billing" in category:
        category = "billing"
    elif "technical" in category:
        category = "technical"
    else:
        category = "general"
    
    print(f"🎯 Supervisor routed to: {category}")
    return {"category": category}


def billing_agent(state: CustomerState) -> dict:
    """Handles billing-related requests."""
    prompt = f"""You are a billing specialist. Help with this request:
    {state['request']}
    
    Be helpful and mention relevant policies."""
    
    response = llm.invoke(prompt)
    print("💳 Billing agent responded")
    return {"response": response.content}


def technical_agent(state: CustomerState) -> dict:
    """Handles technical support requests."""
    prompt = f"""You are a technical support specialist. Help with:
    {state['request']}
    
    Provide clear steps and explanations."""
    
    response = llm.invoke(prompt)
    print("🔧 Technical agent responded")
    return {"response": response.content}


def general_agent(state: CustomerState) -> dict:
    """Handles general inquiries."""
    prompt = f"""You are a customer service representative. Help with:
    {state['request']}
    
    Be friendly and helpful."""
    
    response = llm.invoke(prompt)
    print("💬 General agent responded")
    return {"response": response.content}


def route_to_worker(state: CustomerState) -> Literal["billing", "technical", "general"]:
    """Routes to the appropriate worker based on category."""
    return state["category"]


workflow = StateGraph(CustomerState)

workflow.add_node("supervisor", supervisor)
workflow.add_node("billing", billing_agent)
workflow.add_node("technical", technical_agent)
workflow.add_node("general", general_agent)

# Supervisor first
workflow.add_edge(START, "supervisor")

# Conditional routing based on supervisor's decision
workflow.add_conditional_edges(
    "supervisor",
    route_to_worker,
    {
        "billing": "billing",
        "technical": "technical",
        "general": "general"
    }
)

# All workers go to END
workflow.add_edge("billing", END)
workflow.add_edge("technical", END)
workflow.add_edge("general", END)

app = workflow.compile()

# Test with different types of requests
requests = [
    "I was charged twice for my subscription last month",
    "The app keeps crashing when I try to upload photos",
    "Do you offer student discounts?"
]

for req in requests:
    print(f"\n{'='*50}")
    print(f"REQUEST: {req}")
    print("=" * 50)
    
    result = app.invoke({
        "request": req,
        "category": "",
        "response": ""
    })
    
    print(f"\nRESPONSE:\n{result['response'][:200]}...")


In [ ]:
# From: shared_state_pattern.py

# From: Zero to AI Agent, Chapter 16, Section 16.2
# File: shared_state_pattern.py

"""
Shared State pattern: Agents collaborate via common knowledge base.
"""

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class AnalysisState(TypedDict):
    problem: str
    observations: list[str]   # Shared list all agents can append to
    hypothesis: str
    conclusion: str


def data_collector(state: AnalysisState) -> dict:
    """Gathers initial observations about the problem."""
    prompt = f"""Analyze this problem and list 2-3 key observations:
    
    Problem: {state['problem']}
    
    Format: One observation per line, starting with a dash."""
    
    response = llm.invoke(prompt)
    
    # Parse observations and add to shared list
    new_obs = [line.strip("- ").strip() 
               for line in response.content.split("\n") 
               if line.strip().startswith("-")]
    
    print(f"📊 Collector added {len(new_obs)} observations")
    return {"observations": state["observations"] + new_obs}


def pattern_finder(state: AnalysisState) -> dict:
    """Looks for patterns in collected observations."""
    current_obs = "\n".join(f"- {o}" for o in state["observations"])
    
    prompt = f"""Given these observations, identify 1-2 patterns or connections:
    
    {current_obs}
    
    Format: One pattern per line, starting with a dash."""
    
    response = llm.invoke(prompt)
    
    new_patterns = [line.strip("- ").strip() 
                   for line in response.content.split("\n") 
                   if line.strip().startswith("-")]
    
    print(f"🔍 Pattern finder added {len(new_patterns)} patterns")
    return {"observations": state["observations"] + new_patterns}


def hypothesis_maker(state: AnalysisState) -> dict:
    """Forms a hypothesis based on all observations."""
    all_obs = "\n".join(f"- {o}" for o in state["observations"])
    
    prompt = f"""Based on all these observations and patterns:
    
    {all_obs}
    
    Form a single hypothesis explaining the situation.
    Keep it to one sentence."""
    
    response = llm.invoke(prompt)
    print("💡 Hypothesis formed")
    return {"hypothesis": response.content}


def conclusion_maker(state: AnalysisState) -> dict:
    """Draws final conclusion from hypothesis and observations."""
    all_obs = "\n".join(f"- {o}" for o in state["observations"])
    
    prompt = f"""Given:
    Observations: {all_obs}
    Hypothesis: {state['hypothesis']}
    
    Write a brief conclusion with one recommended action."""
    
    response = llm.invoke(prompt)
    print("✅ Conclusion reached")
    return {"conclusion": response.content}


workflow = StateGraph(AnalysisState)

workflow.add_node("collector", data_collector)
workflow.add_node("pattern_finder", pattern_finder)
workflow.add_node("hypothesis_maker", hypothesis_maker)
workflow.add_node("conclusion_maker", conclusion_maker)

workflow.add_edge(START, "collector")
workflow.add_edge("collector", "pattern_finder")
workflow.add_edge("pattern_finder", "hypothesis_maker")
workflow.add_edge("hypothesis_maker", "conclusion_maker")
workflow.add_edge("conclusion_maker", END)

app = workflow.compile()

# Test collaborative analysis
result = app.invoke({
    "problem": "Our e-commerce site's conversion rate dropped 30% last month",
    "observations": [],
    "hypothesis": "",
    "conclusion": ""
})

print("\n" + "=" * 50)
print("SHARED OBSERVATIONS:")
for i, obs in enumerate(result["observations"], 1):
    print(f"  {i}. {obs}")

print("\nHYPOTHESIS:")
print(f"  {result['hypothesis']}")

print("\nCONCLUSION:")
print(f"  {result['conclusion']}")


---
### Section 16.2 Exercises

### Exercise 16.2.1: Document Processing Pipeline

Build a sequential pipeline for processing documents with these stages:
1. **Extractor** - Pulls out key entities (people, places, dates)
2. **Classifier** - Categorizes the document type (legal, medical, financial, other)
3. **Summarizer** - Creates a summary appropriate for that document type

Test with a sample document of your choice.

In [ ]:
# Your code here


### Exercise 16.2.2: Multi-Perspective Analysis

Create a broadcast pattern for analyzing a business decision. Three agents should provide different perspectives:
1. **Optimist** - Focuses on potential benefits and opportunities
2. **Pessimist** - Focuses on risks and potential problems
3. **Pragmatist** - Focuses on practical implementation concerns

Add an aggregator that synthesizes a balanced recommendation.

In [ ]:
# Your code here


### Exercise 16.2.3: Smart Router with Fallback

Build a hierarchical system that routes questions to specialists:
- **Math Agent** - Handles calculations and math problems
- **History Agent** - Handles historical questions
- **Science Agent** - Handles science questions
- **Fallback Agent** - Handles anything that doesn't fit

The supervisor should route based on question content. Test with at least 5 different questions across categories.

In [ ]:
# Your code here


---
## Section 16.3: Supervisor-worker architectures

In [ ]:
# From: basic_supervisor.py

# From: Zero to AI Agent, Chapter 16, Section 16.3
# File: basic_supervisor.py

"""
A supervisor that coordinates writing specialists.
"""

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class WritingState(TypedDict):
    request: str
    task_type: str
    draft: str
    final_output: str


def email_writer(state: WritingState) -> dict:
    """Specialist for professional emails."""
    prompt = f"""Write a professional email for this request:
    {state['request']}
    
    Use proper email format with subject line, greeting, body, and signature."""
    
    response = llm.invoke(prompt)
    print("📧 Email writer completed")
    return {"draft": response.content}


def blog_writer(state: WritingState) -> dict:
    """Specialist for blog posts."""
    prompt = f"""Write an engaging blog post for this request:
    {state['request']}
    
    Include a catchy title, introduction, main points, and conclusion."""
    
    response = llm.invoke(prompt)
    print("📝 Blog writer completed")
    return {"draft": response.content}


def summary_writer(state: WritingState) -> dict:
    """Specialist for summaries and briefs."""
    prompt = f"""Write a concise summary for this request:
    {state['request']}
    
    Be brief but comprehensive. Use bullet points if helpful."""
    
    response = llm.invoke(prompt)
    print("📋 Summary writer completed")
    return {"draft": response.content}


def supervisor(state: WritingState) -> dict:
    """Analyzes request and decides which specialist to use."""
    prompt = f"""Analyze this writing request and categorize it:
    
    Request: {state['request']}
    
    Categories:
    - email: Professional correspondence, formal messages, business communication
    - blog: Articles, posts, educational content, opinion pieces
    - summary: Condensing information, briefs, overviews, TL;DR
    
    Reply with just the category name (email, blog, or summary)."""
    
    response = llm.invoke(prompt)
    task_type = response.content.strip().lower()
    
    # Normalize the response
    if "email" in task_type:
        task_type = "email"
    elif "blog" in task_type:
        task_type = "blog"
    else:
        task_type = "summary"
    
    print(f"🎯 Supervisor assigned: {task_type}")
    return {"task_type": task_type}


def finalizer(state: WritingState) -> dict:
    """Reviews and finalizes the draft."""
    prompt = f"""Review this {state['task_type']} draft and make minor improvements:
    
    {state['draft']}
    
    Fix any issues but preserve the style. Return the polished version."""
    
    response = llm.invoke(prompt)
    return {"final_output": response.content}


def route_to_worker(state: WritingState) -> Literal["email", "blog", "summary"]:
    """Routes to the appropriate specialist."""
    return state["task_type"]


# Build the graph
workflow = StateGraph(WritingState)

workflow.add_node("supervisor", supervisor)
workflow.add_node("email", email_writer)
workflow.add_node("blog", blog_writer)
workflow.add_node("summary", summary_writer)
workflow.add_node("finalizer", finalizer)

# Supervisor decides first
workflow.add_edge(START, "supervisor")

# Route to appropriate worker
workflow.add_conditional_edges(
    "supervisor",
    route_to_worker,
    {
        "email": "email",
        "blog": "blog",
        "summary": "summary"
    }
)

# All workers go to finalizer
workflow.add_edge("email", "finalizer")
workflow.add_edge("blog", "finalizer")
workflow.add_edge("summary", "finalizer")

workflow.add_edge("finalizer", END)

app = workflow.compile()

# Test it
requests = [
    "Write to my boss asking for a day off next Friday",
    "Create content about the benefits of remote work",
    "Condense this 10-page report into key takeaways"
]

for req in requests:
    print(f"\n{'='*60}")
    print(f"REQUEST: {req[:50]}...")
    print("=" * 60)
    
    result = app.invoke({
        "request": req,
        "task_type": "",
        "draft": "",
        "final_output": ""
    })
    
    print(f"\nOUTPUT:\n{result['final_output'][:300]}...")


In [ ]:
# From: iterative_supervisor.py

# From: Zero to AI Agent, Chapter 16, Section 16.3
# File: iterative_supervisor.py

"""
Supervisor that can call workers multiple times.
"""

from typing import TypedDict, Literal, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import operator

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class ResearchState(TypedDict):
    query: str
    findings: Annotated[list[str], operator.add]  # Accumulates findings
    iteration: int
    max_iterations: int
    needs_more: bool
    final_report: str


def web_researcher(state: ResearchState) -> dict:
    """Simulates web research (in reality, would use search tools)."""
    prompt = f"""For the query: {state['query']}
    
    Previous findings: {state['findings']}
    
    Provide ONE new finding that hasn't been mentioned yet.
    Be specific and factual. Just the finding, no preamble."""
    
    response = llm.invoke(prompt)
    print(f"🔍 Web researcher found: {response.content[:50]}...")
    return {"findings": [f"[Web] {response.content}"]}


def academic_researcher(state: ResearchState) -> dict:
    """Simulates academic research."""
    prompt = f"""For the query: {state['query']}
    
    Previous findings: {state['findings']}
    
    Provide ONE academic or research-based finding not yet mentioned.
    Cite a plausible source. Just the finding, no preamble."""
    
    response = llm.invoke(prompt)
    print(f"📚 Academic researcher found: {response.content[:50]}...")
    return {"findings": [f"[Academic] {response.content}"]}


def research_supervisor(state: ResearchState) -> dict:
    """Decides whether to continue research or compile results."""
    current_iteration = state.get("iteration", 0) + 1
    max_iter = state.get("max_iterations", 3)
    
    # Check if we have enough findings
    if len(state["findings"]) >= 4 or current_iteration > max_iter:
        print(f"📊 Supervisor: Sufficient findings ({len(state['findings'])})")
        return {"iteration": current_iteration, "needs_more": False}
    
    print(f"📊 Supervisor: Need more research (iteration {current_iteration})")
    return {"iteration": current_iteration, "needs_more": True}


def choose_researcher(state: ResearchState) -> Literal["web", "academic"]:
    """Alternates between research sources."""
    # Simple alternation - could be smarter based on query type
    if len(state["findings"]) % 2 == 0:
        return "web"
    return "academic"


def report_compiler(state: ResearchState) -> dict:
    """Compiles all findings into a final report."""
    findings_text = "\n".join(f"- {f}" for f in state["findings"])
    
    prompt = f"""Compile these research findings into a coherent report:
    
    Query: {state['query']}
    
    Findings:
    {findings_text}
    
    Write a 2-3 paragraph summary that synthesizes the findings."""
    
    response = llm.invoke(prompt)
    print("📝 Report compiled")
    return {"final_report": response.content}


def should_continue(state: ResearchState) -> Literal["research", "compile"]:
    """Decides whether to continue researching or compile."""
    if state.get("needs_more", True):
        return "research"
    return "compile"


def route_researcher(state: ResearchState) -> Literal["web", "academic"]:
    """Routes to specific researcher."""
    return choose_researcher(state)


# Build the iterative workflow
workflow = StateGraph(ResearchState)

workflow.add_node("supervisor", research_supervisor)
workflow.add_node("web", web_researcher)
workflow.add_node("academic", academic_researcher)
workflow.add_node("compiler", report_compiler)

workflow.add_edge(START, "supervisor")

# Supervisor decides: more research or compile
workflow.add_conditional_edges(
    "supervisor",
    should_continue,
    {
        "research": "router",  # Go get more findings
        "compile": "compiler"   # Done, compile report
    }
)

# Router node to pick researcher
workflow.add_node("router", lambda s: {})  # Pass-through
workflow.add_conditional_edges(
    "router",
    route_researcher,
    {
        "web": "web",
        "academic": "academic"
    }
)

# Researchers loop back to supervisor
workflow.add_edge("web", "supervisor")
workflow.add_edge("academic", "supervisor")

workflow.add_edge("compiler", END)

app = workflow.compile()

# Test the iterative research
result = app.invoke({
    "query": "What are the health effects of intermittent fasting?",
    "findings": [],
    "iteration": 0,
    "max_iterations": 3,
    "needs_more": True,
    "final_report": ""
})

print("\n" + "=" * 60)
print("FINAL RESEARCH REPORT")
print("=" * 60)
print(f"\nFindings collected: {len(result['findings'])}")
print(f"\nReport:\n{result['final_report']}")


In [ ]:
# From: fault_tolerant_supervisor.py

# From: Zero to AI Agent, Chapter 16, Section 16.3
# File: fault_tolerant_supervisor.py

"""
Supervisor with error handling and fallbacks.
"""

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import random

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class TaskState(TypedDict):
    task: str
    worker_attempts: int
    max_attempts: int
    result: str
    error: str
    status: str  # "pending", "success", "failed"


def unreliable_worker(state: TaskState) -> dict:
    """A worker that fails 40% of the time (for demonstration)."""
    attempts = state.get("worker_attempts", 0) + 1
    
    # Simulate random failures
    if random.random() < 0.4:
        print(f"❌ Worker failed (attempt {attempts})")
        return {
            "worker_attempts": attempts,
            "error": f"Worker failed on attempt {attempts}",
            "status": "pending"
        }
    
    # Success case
    prompt = f"Complete this task briefly: {state['task']}"
    response = llm.invoke(prompt)
    print(f"✅ Worker succeeded (attempt {attempts})")
    
    return {
        "worker_attempts": attempts,
        "result": response.content,
        "error": "",
        "status": "success"
    }


def fallback_worker(state: TaskState) -> dict:
    """Simpler, more reliable fallback."""
    print("🔄 Fallback worker activated")
    
    prompt = f"Provide a simple response for: {state['task']}"
    response = llm.invoke(prompt)
    
    return {
        "result": f"[Fallback] {response.content}",
        "status": "success"
    }


def supervisor_with_retry(state: TaskState) -> dict:
    """Supervisor that manages retries and fallbacks."""
    attempts = state.get("worker_attempts", 0)
    max_attempts = state.get("max_attempts", 3)
    status = state.get("status", "pending")
    
    if status == "success":
        print("📊 Supervisor: Task completed successfully")
    elif attempts >= max_attempts:
        print(f"📊 Supervisor: Max attempts ({max_attempts}) reached, using fallback")
    else:
        print(f"📊 Supervisor: Attempt {attempts + 1} of {max_attempts}")
    
    return {}  # State already updated by worker


def route_after_attempt(state: TaskState) -> Literal["worker", "fallback", "done"]:
    """Decides next step based on worker result."""
    if state.get("status") == "success":
        return "done"
    
    attempts = state.get("worker_attempts", 0)
    max_attempts = state.get("max_attempts", 3)
    
    if attempts >= max_attempts:
        return "fallback"
    
    return "worker"


# Build the fault-tolerant workflow
workflow = StateGraph(TaskState)

workflow.add_node("supervisor", supervisor_with_retry)
workflow.add_node("worker", unreliable_worker)
workflow.add_node("fallback", fallback_worker)

workflow.add_edge(START, "supervisor")
workflow.add_edge("supervisor", "worker")

# After worker, decide what's next
workflow.add_conditional_edges(
    "worker",
    route_after_attempt,
    {
        "worker": "worker",      # Retry
        "fallback": "fallback",  # Give up, use fallback
        "done": END              # Success!
    }
)

workflow.add_edge("fallback", END)

app = workflow.compile()

# Test reliability
print("Testing fault-tolerant supervisor (results will vary):\n")

for i in range(3):
    print(f"\n{'='*50}")
    print(f"Test run {i+1}")
    print("=" * 50)
    
    result = app.invoke({
        "task": "Explain what Python is in one sentence",
        "worker_attempts": 0,
        "max_attempts": 3,
        "result": "",
        "error": "",
        "status": "pending"
    })
    
    print(f"\nFinal result: {result['result'][:100]}...")
    print(f"Attempts used: {result['worker_attempts']}")


In [ ]:
# From: supervisor_hierarchy.py

# From: Zero to AI Agent, Chapter 16, Section 16.3
# File: supervisor_hierarchy.py

"""
Two-level supervisor hierarchy for content creation.
"""

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


class ContentState(TypedDict):
    topic: str
    stage: str  # "research", "writing", "review", "done"
    research_notes: str
    draft: str
    review_feedback: str
    final_content: str


def research_worker(state: ContentState) -> dict:
    """Gathers information on the topic."""
    prompt = f"List 3 key facts about: {state['topic']}"
    response = llm.invoke(prompt)
    print("  📚 Research worker complete")
    return {"research_notes": response.content}


def writing_worker(state: ContentState) -> dict:
    """Writes content based on research."""
    prompt = f"""Write a short piece about {state['topic']}.
    Use these notes: {state['research_notes']}"""
    response = llm.invoke(prompt)
    print("  ✍️ Writing worker complete")
    return {"draft": response.content}


def review_worker(state: ContentState) -> dict:
    """Reviews and provides feedback."""
    prompt = f"""Review this draft and provide 2 improvement suggestions:
    {state['draft']}"""
    response = llm.invoke(prompt)
    print("  🔍 Review worker complete")
    return {"review_feedback": response.content}


def polish_worker(state: ContentState) -> dict:
    """Applies feedback to create final version."""
    prompt = f"""Improve this draft based on feedback:
    
    Draft: {state['draft']}
    Feedback: {state['review_feedback']}"""
    response = llm.invoke(prompt)
    print("  ✨ Polish worker complete")
    return {"final_content": response.content}


def top_supervisor(state: ContentState) -> dict:
    """High-level supervisor that manages the content pipeline."""
    current_stage = state.get("stage", "research")
    
    stage_order = ["research", "writing", "review", "polish", "done"]
    current_index = stage_order.index(current_stage)
    
    if current_index < len(stage_order) - 1:
        next_stage = stage_order[current_index + 1]
    else:
        next_stage = "done"
    
    print(f"🎯 Top supervisor: {current_stage} → {next_stage}")
    return {"stage": next_stage}


def route_by_stage(state: ContentState) -> Literal["research", "writing", "review", "polish", "done"]:
    """Routes to appropriate stage."""
    return state["stage"]


# Build the hierarchy
workflow = StateGraph(ContentState)

workflow.add_node("top_supervisor", top_supervisor)
workflow.add_node("research", research_worker)
workflow.add_node("writing", writing_worker)
workflow.add_node("review", review_worker)
workflow.add_node("polish", polish_worker)

# Start with supervisor
workflow.add_edge(START, "top_supervisor")

# Route based on stage
workflow.add_conditional_edges(
    "top_supervisor",
    route_by_stage,
    {
        "research": "research",
        "writing": "writing",
        "review": "review",
        "polish": "polish",
        "done": END
    }
)

# Each worker returns to supervisor for next decision
workflow.add_edge("research", "top_supervisor")
workflow.add_edge("writing", "top_supervisor")
workflow.add_edge("review", "top_supervisor")
workflow.add_edge("polish", "top_supervisor")

app = workflow.compile()

# Run the pipeline
result = app.invoke({
    "topic": "The future of renewable energy",
    "stage": "research",
    "research_notes": "",
    "draft": "",
    "review_feedback": "",
    "final_content": ""
})

print("\n" + "=" * 60)
print("FINAL CONTENT")
print("=" * 60)
print(result["final_content"])


---
### Section 16.3 Exercises

### Exercise 16.3.1: Customer Service Supervisor

Build a supervisor-worker system for customer service with these workers:
- **Greeter** - Welcomes customer and classifies their issue
- **Billing Agent** - Handles payment and subscription issues
- **Tech Support** - Handles technical problems
- **Complaint Handler** - Handles complaints and escalations

The supervisor should:
1. Route to the appropriate specialist
2. If the specialist can't fully resolve, route to a human handoff message
3. Track the number of routing decisions made

In [ ]:
# Your code here


### Exercise 16.3.2: Document Processing Pipeline

Create an iterative supervisor for document processing:
- **Extractor** - Pulls out key information
- **Validator** - Checks extracted info for completeness
- **Enricher** - Adds additional context

The supervisor should:
1. Run extractor first
2. Check validator—if incomplete, run extractor again (max 2 retries)
3. Only proceed to enricher when validator passes
4. Compile final structured document

In [ ]:
# Your code here


### Exercise 16.3.3: Quality Control Supervisor

Build a supervisor with quality control loops:
- **Writer** - Creates content
- **Critic** - Scores content 1-10 with feedback
- **Improver** - Revises based on feedback

The supervisor should:
1. Start with writer
2. Get critic score
3. If score \< 7, send to improver, then back to critic
4. Loop until score \>= 7 or max 3 improvement rounds
5. Return final content with score history

In [ ]:
# Your code here


---
## Section 16.4: Collaborative agent systems

In [ ]:
# From: debate_system.py

# From: Zero to AI Agent, Chapter 16, Section 16.4
# File: debate_system.py

"""
Collaborative debate between agents with opposing viewpoints.
"""

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import operator

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


class DebateState(TypedDict):
    topic: str
    pro_arguments: Annotated[list[str], operator.add]
    con_arguments: Annotated[list[str], operator.add]
    current_round: int
    max_rounds: int
    synthesis: str


def pro_debater(state: DebateState) -> dict:
    """Argues in favor of the topic."""
    existing_pro = state.get("pro_arguments", [])
    existing_con = state.get("con_arguments", [])
    round_num = state.get("current_round", 1)
    
    prompt = f"""You are arguing IN FAVOR of: {state['topic']}
    
    Round {round_num} of the debate.
    
    Previous PRO arguments: {existing_pro}
    Previous CON arguments: {existing_con}
    
    Provide ONE new compelling argument for your position.
    If there are CON arguments, you may also rebut them.
    Be concise but persuasive (2-3 sentences)."""
    
    response = llm.invoke(prompt)
    print(f"✅ PRO (Round {round_num}): {response.content[:60]}...")
    
    return {"pro_arguments": [f"[R{round_num}] {response.content}"]}


def con_debater(state: DebateState) -> dict:
    """Argues against the topic."""
    existing_pro = state.get("pro_arguments", [])
    existing_con = state.get("con_arguments", [])
    round_num = state.get("current_round", 1)
    
    prompt = f"""You are arguing AGAINST: {state['topic']}
    
    Round {round_num} of the debate.
    
    Previous PRO arguments: {existing_pro}
    Previous CON arguments: {existing_con}
    
    Provide ONE new compelling argument against the position.
    You may also rebut PRO arguments.
    Be concise but persuasive (2-3 sentences)."""
    
    response = llm.invoke(prompt)
    print(f"❌ CON (Round {round_num}): {response.content[:60]}...")
    
    return {"con_arguments": [f"[R{round_num}] {response.content}"]}


def round_coordinator(state: DebateState) -> dict:
    """Advances to the next round."""
    current = state.get("current_round", 0)
    return {"current_round": current + 1}


def judge(state: DebateState) -> dict:
    """Synthesizes arguments into a balanced conclusion."""
    pro_args = "\n".join(state.get("pro_arguments", []))
    con_args = "\n".join(state.get("con_arguments", []))
    
    prompt = f"""As an impartial judge, synthesize this debate:
    
    TOPIC: {state['topic']}
    
    ARGUMENTS IN FAVOR:
    {pro_args}
    
    ARGUMENTS AGAINST:
    {con_args}
    
    Provide:
    1. The strongest point from each side
    2. A balanced conclusion
    3. What additional information would help decide
    
    Be fair and analytical."""
    
    response = llm.invoke(prompt)
    print("⚖️ Judge has reached a conclusion")
    
    return {"synthesis": response.content}


def should_continue_debate(state: DebateState) -> str:
    """Checks if debate should continue."""
    current = state.get("current_round", 0)
    max_rounds = state.get("max_rounds", 2)
    
    if current >= max_rounds:
        return "judge"
    return "continue"


# Build the debate workflow
workflow = StateGraph(DebateState)

workflow.add_node("coordinator", round_coordinator)
workflow.add_node("pro", pro_debater)
workflow.add_node("con", con_debater)
workflow.add_node("judge", judge)

# Start with coordinator to set round 1
workflow.add_edge(START, "coordinator")

# After coordinator, check if we should continue
workflow.add_conditional_edges(
    "coordinator",
    should_continue_debate,
    {
        "continue": "pro",
        "judge": "judge"
    }
)

# Pro and Con take turns (both run each round)
workflow.add_edge("pro", "con")
workflow.add_edge("con", "coordinator")  # Back to coordinator for next round

workflow.add_edge("judge", END)

app = workflow.compile()

# Run a debate
result = app.invoke({
    "topic": "Remote work should be the default for knowledge workers",
    "pro_arguments": [],
    "con_arguments": [],
    "current_round": 0,
    "max_rounds": 2,
    "synthesis": ""
})

print("\n" + "=" * 60)
print("DEBATE CONCLUSION")
print("=" * 60)
print(result["synthesis"])


In [ ]:
# From: critique_refine.py

# From: Zero to AI Agent, Chapter 16, Section 16.4
# File: critique_refine.py

"""
Collaborative critique and refinement between agents.
"""

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import operator

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


class CritiqueState(TypedDict):
    task: str
    current_work: str
    critique_history: Annotated[list[str], operator.add]
    revision_count: int
    max_revisions: int
    is_approved: bool
    final_work: str


def creator(state: CritiqueState) -> dict:
    """Creates or revises the work based on critique."""
    revision = state.get("revision_count", 0)
    critique_history = state.get("critique_history", [])
    current = state.get("current_work", "")
    
    if revision == 0:
        # Initial creation
        prompt = f"""Create a response for this task:
        {state['task']}
        
        Be thorough but concise."""
    else:
        # Revision based on critique
        latest_critique = critique_history[-1] if critique_history else ""
        prompt = f"""Revise your work based on this critique:
        
        TASK: {state['task']}
        
        YOUR PREVIOUS WORK:
        {current}
        
        CRITIQUE:
        {latest_critique}
        
        Address the concerns while keeping what works."""
    
    response = llm.invoke(prompt)
    action = "Created" if revision == 0 else f"Revised (v{revision + 1})"
    print(f"✍️ Creator: {action}")
    
    return {
        "current_work": response.content,
        "revision_count": revision + 1
    }


def critic(state: CritiqueState) -> dict:
    """Evaluates the work and provides constructive critique."""
    prompt = f"""Evaluate this work critically but constructively:
    
    TASK: {state['task']}
    
    WORK TO EVALUATE:
    {state['current_work']}
    
    Provide:
    1. What works well (be specific)
    2. What needs improvement (be specific)  
    3. VERDICT: APPROVE if good enough, or REVISE if needs work
    
    Be fair but maintain high standards."""
    
    response = llm.invoke(prompt)
    
    is_approved = "APPROVE" in response.content.upper() and "REVISE" not in response.content.upper()
    
    verdict = "APPROVED ✓" if is_approved else "NEEDS REVISION"
    print(f"🔍 Critic: {verdict}")
    
    return {
        "critique_history": [response.content],
        "is_approved": is_approved
    }


def should_revise(state: CritiqueState) -> str:
    """Decides whether to revise or finalize."""
    if state.get("is_approved", False):
        return "finalize"
    
    revisions = state.get("revision_count", 0)
    max_rev = state.get("max_revisions", 3)
    
    if revisions >= max_rev:
        print(f"⚠️ Max revisions ({max_rev}) reached")
        return "finalize"
    
    return "revise"


def finalizer(state: CritiqueState) -> dict:
    """Packages the final approved work."""
    return {"final_work": state["current_work"]}


# Build the collaborative workflow
workflow = StateGraph(CritiqueState)

workflow.add_node("creator", creator)
workflow.add_node("critic", critic)
workflow.add_node("finalizer", finalizer)

workflow.add_edge(START, "creator")
workflow.add_edge("creator", "critic")

workflow.add_conditional_edges(
    "critic",
    should_revise,
    {
        "revise": "creator",
        "finalize": "finalizer"
    }
)

workflow.add_edge("finalizer", END)

app = workflow.compile()

# Test the critique cycle
result = app.invoke({
    "task": "Write a professional bio for a software engineer with 5 years experience",
    "current_work": "",
    "critique_history": [],
    "revision_count": 0,
    "max_revisions": 3,
    "is_approved": False,
    "final_work": ""
})

print("\n" + "=" * 60)
print(f"FINAL WORK (after {result['revision_count']} revisions)")
print("=" * 60)
print(result["final_work"])
print("\n" + "=" * 60)
print("CRITIQUE HISTORY")
print("=" * 60)
for i, critique in enumerate(result["critique_history"], 1):
    print(f"\n--- Critique {i} ---")
    print(critique[:200] + "...")


In [ ]:
# From: ensemble_system.py

# From: Zero to AI Agent, Chapter 16, Section 16.4
# File: ensemble_system.py

"""
Ensemble pattern: Multiple agents solve independently, then merge.
"""

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)  # Higher temp for diversity


class EnsembleState(TypedDict):
    problem: str
    solution_creative: str
    solution_analytical: str
    solution_practical: str
    merged_solution: str


def creative_solver(state: EnsembleState) -> dict:
    """Approaches problem creatively."""
    prompt = f"""Solve this problem with CREATIVE thinking:
    
    {state['problem']}
    
    Think outside the box. Consider unconventional approaches.
    Be imaginative but still address the core problem."""
    
    response = llm.invoke(prompt)
    print("🎨 Creative solver complete")
    return {"solution_creative": response.content}


def analytical_solver(state: EnsembleState) -> dict:
    """Approaches problem analytically."""
    prompt = f"""Solve this problem with ANALYTICAL thinking:
    
    {state['problem']}
    
    Break it down systematically. Consider data and logic.
    Be thorough and evidence-based."""
    
    response = llm.invoke(prompt)
    print("📊 Analytical solver complete")
    return {"solution_analytical": response.content}


def practical_solver(state: EnsembleState) -> dict:
    """Approaches problem practically."""
    prompt = f"""Solve this problem with PRACTICAL thinking:
    
    {state['problem']}
    
    Focus on what's actionable and realistic.
    Consider constraints and implementation."""
    
    response = llm.invoke(prompt)
    print("🔧 Practical solver complete")
    return {"solution_practical": response.content}


def solution_merger(state: EnsembleState) -> dict:
    """Synthesizes the three approaches into one solution."""
    prompt = f"""Synthesize these three approaches to the problem:
    
    PROBLEM: {state['problem']}
    
    CREATIVE APPROACH:
    {state['solution_creative']}
    
    ANALYTICAL APPROACH:
    {state['solution_analytical']}
    
    PRACTICAL APPROACH:
    {state['solution_practical']}
    
    Create a unified solution that:
    1. Takes the best ideas from each approach
    2. Resolves any contradictions
    3. Is both innovative AND actionable
    
    Provide a clear, integrated recommendation."""
    
    response = llm.invoke(prompt)
    print("🔀 Solutions merged")
    return {"merged_solution": response.content}


# Build with parallel execution
workflow = StateGraph(EnsembleState)

workflow.add_node("creative", creative_solver)
workflow.add_node("analytical", analytical_solver)
workflow.add_node("practical", practical_solver)
workflow.add_node("merger", solution_merger)

# All three solvers start in parallel
workflow.add_edge(START, "creative")
workflow.add_edge(START, "analytical")
workflow.add_edge(START, "practical")

# All feed into merger
workflow.add_edge("creative", "merger")
workflow.add_edge("analytical", "merger")
workflow.add_edge("practical", "merger")

workflow.add_edge("merger", END)

app = workflow.compile()

# Test ensemble solving
result = app.invoke({
    "problem": """Our startup needs to acquire our first 1000 customers 
    with a limited budget of $5000. We're a B2B SaaS tool for project management.""",
    "solution_creative": "",
    "solution_analytical": "",
    "solution_practical": "",
    "merged_solution": ""
})

print("\n" + "=" * 60)
print("ENSEMBLE SOLUTION")
print("=" * 60)
print(result["merged_solution"])


In [ ]:
# From: consensus_system.py

# From: Zero to AI Agent, Chapter 16, Section 16.4
# File: consensus_system.py

"""
Consensus building among multiple agents.
"""

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import operator

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)


class ConsensusState(TypedDict):
    question: str
    proposals: Annotated[list[str], operator.add]
    discussion: Annotated[list[str], operator.add]
    round: int
    max_rounds: int
    consensus_reached: bool
    final_decision: str


def agent_alpha(state: ConsensusState) -> dict:
    """First perspective agent."""
    round_num = state.get("round", 1)
    existing_proposals = state.get("proposals", [])
    existing_discussion = state.get("discussion", [])
    
    if round_num == 1:
        # Initial proposal
        prompt = f"""Propose your answer to: {state['question']}
        
        Give a clear, concise recommendation with brief reasoning."""
    else:
        # Respond to discussion
        prompt = f"""Question: {state['question']}
        
        Previous proposals: {existing_proposals}
        Discussion so far: {existing_discussion}
        
        Based on the discussion, do you:
        1. Maintain your position (explain why)
        2. Modify your position (explain what changed)
        3. Accept another proposal (say which one and why)
        
        Be constructive and aim for consensus."""
    
    response = llm.invoke(prompt)
    
    if round_num == 1:
        return {"proposals": [f"[Alpha] {response.content}"]}
    return {"discussion": [f"[Alpha R{round_num}] {response.content}"]}


def agent_beta(state: ConsensusState) -> dict:
    """Second perspective agent."""
    round_num = state.get("round", 1)
    existing_proposals = state.get("proposals", [])
    existing_discussion = state.get("discussion", [])
    
    if round_num == 1:
        prompt = f"""Propose your answer to: {state['question']}
        
        Consider a different angle than others might take.
        Give a clear recommendation with reasoning."""
    else:
        prompt = f"""Question: {state['question']}
        
        Previous proposals: {existing_proposals}
        Discussion so far: {existing_discussion}
        
        Respond constructively. State if you agree, disagree, or want to modify.
        Focus on finding common ground."""
    
    response = llm.invoke(prompt)
    
    if round_num == 1:
        return {"proposals": [f"[Beta] {response.content}"]}
    return {"discussion": [f"[Beta R{round_num}] {response.content}"]}


def facilitator(state: ConsensusState) -> dict:
    """Checks if consensus has been reached."""
    proposals = state.get("proposals", [])
    discussion = state.get("discussion", [])
    round_num = state.get("round", 0) + 1
    
    if round_num == 1:
        # Just starting, no consensus check yet
        print(f"🗣️ Round {round_num}: Gathering initial proposals")
        return {"round": round_num, "consensus_reached": False}
    
    # Analyze discussion for consensus
    all_content = " ".join(proposals + discussion).lower()
    
    # Simple consensus detection (in production, use LLM for this)
    agreement_signals = ["agree", "accept", "consensus", "common ground", "align"]
    agreement_count = sum(1 for signal in agreement_signals if signal in all_content)
    
    consensus = agreement_count >= 2 or round_num > state.get("max_rounds", 3)
    
    if consensus:
        print(f"✅ Consensus detected after {round_num} rounds")
    else:
        print(f"🗣️ Round {round_num}: Continuing discussion")
    
    return {"round": round_num, "consensus_reached": consensus}


def should_continue(state: ConsensusState) -> str:
    """Decides if discussion should continue."""
    if state.get("consensus_reached", False):
        return "synthesize"
    if state.get("round", 0) >= state.get("max_rounds", 3):
        return "synthesize"
    return "discuss"


def synthesizer(state: ConsensusState) -> dict:
    """Synthesizes discussion into final decision."""
    prompt = f"""Synthesize this group discussion into a final decision:
    
    QUESTION: {state['question']}
    
    INITIAL PROPOSALS:
    {chr(10).join(state.get('proposals', []))}
    
    DISCUSSION:
    {chr(10).join(state.get('discussion', []))}
    
    Provide:
    1. The consensus decision (what the group agreed on)
    2. Key points that led to agreement
    3. Any remaining concerns or caveats"""
    
    response = llm.invoke(prompt)
    print("📋 Final decision synthesized")
    
    return {"final_decision": response.content}


# Build the consensus workflow
workflow = StateGraph(ConsensusState)

workflow.add_node("facilitator", facilitator)
workflow.add_node("alpha", agent_alpha)
workflow.add_node("beta", agent_beta)
workflow.add_node("synthesizer", synthesizer)

workflow.add_edge(START, "facilitator")

workflow.add_conditional_edges(
    "facilitator",
    should_continue,
    {
        "discuss": "alpha",
        "synthesize": "synthesizer"
    }
)

# Discussion flow: alpha -> beta -> facilitator
workflow.add_edge("alpha", "beta")
workflow.add_edge("beta", "facilitator")

workflow.add_edge("synthesizer", END)

app = workflow.compile()

# Test consensus building
result = app.invoke({
    "question": "What programming language should a beginner learn first?",
    "proposals": [],
    "discussion": [],
    "round": 0,
    "max_rounds": 3,
    "consensus_reached": False,
    "final_decision": ""
})

print("\n" + "=" * 60)
print("CONSENSUS DECISION")
print("=" * 60)
print(result["final_decision"])


---
### Section 16.4 Exercises

### Exercise 16.4.1: Three-Way Debate

Extend the debate pattern to include three agents:
- **Optimist** - Focuses on opportunities and benefits
- **Pessimist** - Focuses on risks and problems
- **Realist** - Tries to find middle ground

Each agent should respond to the others' arguments. The judge should identify where all three perspectives align.

In [ ]:
# Your code here


### Exercise 16.4.2: Review Committee

Build a collaborative review system with three reviewers:
- **Technical Reviewer** - Checks accuracy and technical correctness
- **Style Reviewer** - Checks clarity and readability
- **Audience Reviewer** - Checks if it's appropriate for target audience

They should each provide feedback, then collaboratively create a consolidated review with prioritized recommendations.

In [ ]:
# Your code here


### Exercise 16.4.3: Negotiation Simulation

Create two agents that negotiate a deal:
- **Buyer Agent** - Wants lowest price, best terms
- **Seller Agent** - Wants highest price, favorable terms

They should:
1. Each state their initial position
2. Exchange counter-offers (max 3 rounds)
3. Try to find a mutually acceptable deal
4. Report final outcome (deal reached or impasse)

Track concessions made by each side.

In [ ]:
# Your code here


---
## Section 16.5: Building a research assistant team

In [ ]:
# From: research_team.py

# From: Zero to AI Agent, Chapter 16, Section 16.5
# File: research_team.py

"""
A multi-agent research assistant team.
Basic sequential version: Planner -> Researcher -> Analyst -> Writer
"""

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# We use a lower temperature for more focused, factual responses
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)


class ResearchState(TypedDict):
    topic: str                    # The research topic
    questions: list[str]          # Research questions from planner
    findings: list[str]           # Raw findings from researcher
    insights: str                 # Analysis from analyst
    report: str                   # Final report from writer
    status: str                   # Current status for tracking


def planner_agent(state: ResearchState) -> dict:
    """
    Takes a topic and creates focused research questions.
    Good research starts with good questions.
    """
    prompt = f"""You are a research planner. Given a topic, create exactly 3 
focused research questions that would help someone understand it thoroughly.

TOPIC: {state['topic']}

Requirements:
- Each question should cover a different aspect
- Questions should be specific and answerable
- Together they should provide comprehensive coverage

Format your response as:
1. [First question]
2. [Second question]
3. [Third question]"""

    response = llm.invoke(prompt)
    
    # Parse the questions from the response
    lines = response.content.strip().split('\n')
    questions = []
    for line in lines:
        # Remove numbering and clean up
        cleaned = line.strip()
        if cleaned and cleaned[0].isdigit():
            # Remove "1. " or "1) " prefix
            cleaned = cleaned[2:].strip() if len(cleaned) > 2 else cleaned
            if cleaned.startswith('.') or cleaned.startswith(')'):
                cleaned = cleaned[1:].strip()
            questions.append(cleaned)
    
    # Ensure we have exactly 3 questions
    questions = questions[:3] if len(questions) >= 3 else questions
    
    print(f"📋 Planner created {len(questions)} research questions")
    for i, q in enumerate(questions, 1):
        print(f"   {i}. {q[:60]}...")
    
    return {
        "questions": questions,
        "status": "questions_ready"
    }


def researcher_agent(state: ResearchState) -> dict:
    """
    Researches each question and gathers findings.
    In production, this might call search APIs or databases.
    """
    findings = []
    
    for i, question in enumerate(state['questions'], 1):
        prompt = f"""You are a research assistant. Answer this research question 
with factual, specific information.

QUESTION: {question}

Provide 2-3 key facts or findings. Be specific and informative.
Keep your response to 3-4 sentences."""

        response = llm.invoke(prompt)
        finding = f"Q{i}: {question}\nFindings: {response.content}"
        findings.append(finding)
        
        print(f"🔍 Researched question {i}/{len(state['questions'])}")
    
    return {
        "findings": findings,
        "status": "research_complete"
    }


def analyst_agent(state: ResearchState) -> dict:
    """
    Analyzes findings to extract key insights and patterns.
    Moves from raw facts to deeper understanding.
    """
    # Combine all findings into one text block
    all_findings = "\n\n".join(state['findings'])
    
    prompt = f"""You are a research analyst. Review these research findings and 
extract key insights.

TOPIC: {state['topic']}

FINDINGS:
{all_findings}

Provide your analysis:
1. What are the 2-3 most important takeaways?
2. Are there any patterns or connections between findings?
3. What's the overall picture that emerges?

Be concise but insightful."""

    response = llm.invoke(prompt)
    
    print("🔬 Analysis complete")
    
    return {
        "insights": response.content,
        "status": "analysis_complete"
    }


def writer_agent(state: ResearchState) -> dict:
    """
    Compiles all research into a readable final report.
    The output the user actually sees.
    """
    all_findings = "\n\n".join(state['findings'])
    
    prompt = f"""You are a research report writer. Create a clear, well-organized 
report based on this research.

TOPIC: {state['topic']}

RESEARCH FINDINGS:
{all_findings}

KEY INSIGHTS:
{state['insights']}

Write a report with:
1. A brief introduction (2-3 sentences)
2. Main findings organized by theme (use bullet points)
3. A conclusion with key takeaways (2-3 sentences)

Keep it concise but comprehensive. Use clear, professional language."""

    response = llm.invoke(prompt)
    
    print("✍️ Report written")
    
    return {
        "report": response.content,
        "status": "complete"
    }


# Build the research workflow
workflow = StateGraph(ResearchState)

# Add all our agents as nodes
workflow.add_node("planner", planner_agent)
workflow.add_node("researcher", researcher_agent)
workflow.add_node("analyst", analyst_agent)
workflow.add_node("writer", writer_agent)

# Connect them in sequence
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "writer")
workflow.add_edge("writer", END)

# Compile the graph
research_team = workflow.compile()


def run_research(topic: str) -> str:
    """Run the research team on a topic and return the report."""
    
    print("=" * 60)
    print(f"RESEARCHING: {topic}")
    print("=" * 60 + "\n")
    
    result = research_team.invoke({
        "topic": topic,
        "questions": [],
        "findings": [],
        "insights": "",
        "report": "",
        "status": "started"
    })
    
    return result["report"]


# Test it out!
if __name__ == "__main__":
    topic = "The impact of artificial intelligence on healthcare"
    
    report = run_research(topic)
    
    print("\n" + "=" * 60)
    print("FINAL RESEARCH REPORT")
    print("=" * 60)
    print(report)


In [ ]:
# From: research_team_with_review.py

# From: Zero to AI Agent, Chapter 16, Section 16.5
# File: research_team_with_review.py

"""
Research team with quality review loop.
Adds a reviewer that can request revisions before final output.
"""

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)


class ResearchStateWithReview(TypedDict):
    topic: str
    questions: list[str]
    findings: list[str]
    insights: str
    report: str
    review_feedback: str
    revision_count: int
    max_revisions: int
    approved: bool


def planner_agent(state: ResearchStateWithReview) -> dict:
    """Creates research questions from topic."""
    prompt = f"""You are a research planner. Given a topic, create exactly 3 
focused research questions.

TOPIC: {state['topic']}

Format: numbered list, one question per line."""
    
    response = llm.invoke(prompt)
    lines = response.content.strip().split('\n')
    questions = []
    for line in lines:
        cleaned = line.strip()
        if cleaned and cleaned[0].isdigit():
            cleaned = cleaned[2:].strip() if len(cleaned) > 2 else cleaned
            if cleaned.startswith('.') or cleaned.startswith(')'):
                cleaned = cleaned[1:].strip()
            questions.append(cleaned)
    
    questions = questions[:3]
    print(f"📋 Planner created {len(questions)} research questions")
    return {"questions": questions}


def researcher_agent(state: ResearchStateWithReview) -> dict:
    """Researches each question and gathers findings."""
    findings = []
    
    for i, question in enumerate(state['questions'], 1):
        prompt = f"""Answer this research question with 2-3 key facts:

QUESTION: {question}

Be specific and informative. Keep to 3-4 sentences."""

        response = llm.invoke(prompt)
        finding = f"Q{i}: {question}\nFindings: {response.content}"
        findings.append(finding)
        print(f"🔍 Researched question {i}/{len(state['questions'])}")
    
    return {"findings": findings}


def analyst_agent(state: ResearchStateWithReview) -> dict:
    """Analyzes findings to extract key insights."""
    all_findings = "\n\n".join(state['findings'])
    
    prompt = f"""Analyze these research findings:

TOPIC: {state['topic']}

FINDINGS:
{all_findings}

What are the 2-3 most important takeaways? Be concise but insightful."""

    response = llm.invoke(prompt)
    print("🔬 Analysis complete")
    return {"insights": response.content}


def writer_with_revision(state: ResearchStateWithReview) -> dict:
    """Writes or revises the report based on feedback."""
    all_findings = "\n\n".join(state['findings'])
    
    if state.get('review_feedback'):
        # This is a revision
        prompt = f"""Revise this research report based on feedback:

CURRENT REPORT:
{state['report']}

FEEDBACK:
{state['review_feedback']}

Provide an improved version addressing the feedback."""
    else:
        # Initial write
        prompt = f"""Write a research report on: {state['topic']}

FINDINGS:
{all_findings}

INSIGHTS:
{state['insights']}

Include: introduction, main findings, and conclusion."""

    response = llm.invoke(prompt)
    print("✍️ Report " + ("revised" if state.get('review_feedback') else "written"))
    
    return {"report": response.content}


def reviewer_agent(state: ResearchStateWithReview) -> dict:
    """Reviews the report for quality and completeness."""
    prompt = f"""You are a research report reviewer. Evaluate this report:

TOPIC: {state['topic']}

REPORT:
{state['report']}

Check for:
1. Does it address the topic adequately?
2. Is it well-organized and clear?
3. Are claims supported by the findings?

If the report is good, respond with: APPROVED

If it needs improvement, respond with: REVISE: [specific feedback]

Be reasonable - don't demand perfection."""

    response = llm.invoke(prompt)
    content = response.content.strip()
    
    approved = content.upper().startswith("APPROVED")
    revision_count = state.get("revision_count", 0)
    
    if approved:
        print("✅ Report approved!")
    else:
        print(f"📝 Revision requested (attempt {revision_count + 1})")
    
    return {
        "approved": approved,
        "review_feedback": content if not approved else "",
        "revision_count": revision_count + 1
    }


def should_revise(state: ResearchStateWithReview) -> Literal["revise", "done"]:
    """Decide whether to revise or finish."""
    if state.get("approved", False):
        return "done"
    
    max_rev = state.get("max_revisions", 2)
    if state.get("revision_count", 0) >= max_rev:
        print("⚠️ Max revisions reached, accepting current version")
        return "done"
    
    return "revise"


# Build the enhanced workflow
workflow = StateGraph(ResearchStateWithReview)

workflow.add_node("planner", planner_agent)
workflow.add_node("researcher", researcher_agent)
workflow.add_node("analyst", analyst_agent)
workflow.add_node("writer", writer_with_revision)
workflow.add_node("reviewer", reviewer_agent)

# Main flow
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "writer")
workflow.add_edge("writer", "reviewer")

# Review loop
workflow.add_conditional_edges(
    "reviewer",
    should_revise,
    {
        "revise": "writer",
        "done": END
    }
)

research_team_v2 = workflow.compile()


if __name__ == "__main__":
    result = research_team_v2.invoke({
        "topic": "Benefits of meditation for stress reduction",
        "questions": [],
        "findings": [],
        "insights": "",
        "report": "",
        "review_feedback": "",
        "revision_count": 0,
        "max_revisions": 2,
        "approved": False
    })
    
    print("\n" + "=" * 60)
    print("FINAL REPORT")
    print("=" * 60)
    print(result["report"])


In [ ]:
# From: research_team_complete.py

# From: Zero to AI Agent, Chapter 16, Section 16.5
# File: research_team_complete.py

"""
Complete multi-agent research assistant team.
Combines sequential pipeline with quality review loop.
"""

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)


# === STATE ===

class ResearchState(TypedDict):
    topic: str
    questions: list[str]
    findings: list[str]
    insights: str
    report: str
    feedback: str
    revision_count: int
    approved: bool


# === AGENTS ===

def planner(state: ResearchState) -> dict:
    """Creates research questions from topic."""
    prompt = f"""Create 3 research questions for: {state['topic']}
    
Format: numbered list, one question per line."""
    
    response = llm.invoke(prompt)
    questions = [line.strip()[3:] for line in response.content.split('\n') 
                 if line.strip() and line.strip()[0].isdigit()][:3]
    
    print(f"📋 Planner: {len(questions)} questions created")
    return {"questions": questions}


def researcher(state: ResearchState) -> dict:
    """Gathers findings for each question."""
    findings = []
    for i, q in enumerate(state['questions'], 1):
        prompt = f"Answer briefly with 2-3 facts: {q}"
        response = llm.invoke(prompt)
        findings.append(f"Q: {q}\nA: {response.content}")
        print(f"🔍 Researcher: question {i} done")
    return {"findings": findings}


def analyst(state: ResearchState) -> dict:
    """Extracts insights from findings."""
    prompt = f"""Analyze these findings and give 3 key insights:
    
{chr(10).join(state['findings'])}"""
    
    response = llm.invoke(prompt)
    print("🔬 Analyst: insights extracted")
    return {"insights": response.content}


def writer(state: ResearchState) -> dict:
    """Writes or revises the report."""
    if state.get('feedback'):
        prompt = f"""Revise this report based on feedback:
        
REPORT: {state['report']}

FEEDBACK: {state['feedback']}"""
    else:
        prompt = f"""Write a short report on: {state['topic']}

INSIGHTS: {state['insights']}

Include intro, findings, conclusion."""
    
    response = llm.invoke(prompt)
    action = "revised" if state.get('feedback') else "written"
    print(f"✍️ Writer: report {action}")
    return {"report": response.content, "revision_count": state.get('revision_count', 0) + 1}


def reviewer(state: ResearchState) -> dict:
    """Reviews report quality."""
    prompt = f"""Review this report. Reply APPROVED if good, or REVISE: [feedback] if not.

{state['report']}"""
    
    response = llm.invoke(prompt)
    approved = "APPROVED" in response.content.upper()
    print(f"{'✅ Approved' if approved else '📝 Needs revision'}")
    
    return {
        "approved": approved,
        "feedback": "" if approved else response.content
    }


def should_continue(state: ResearchState) -> Literal["revise", "done"]:
    """Check if we should revise or finish."""
    if state.get("approved") or state.get("revision_count", 0) >= 2:
        return "done"
    return "revise"


# === BUILD GRAPH ===

workflow = StateGraph(ResearchState)

workflow.add_node("planner", planner)
workflow.add_node("researcher", researcher)
workflow.add_node("analyst", analyst)
workflow.add_node("writer", writer)
workflow.add_node("reviewer", reviewer)

workflow.add_edge(START, "planner")
workflow.add_edge("planner", "researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "writer")
workflow.add_edge("writer", "reviewer")

workflow.add_conditional_edges("reviewer", should_continue, {
    "revise": "writer",
    "done": END
})

research_team = workflow.compile()


# === RUN ===

if __name__ == "__main__":
    result = research_team.invoke({
        "topic": "Benefits of meditation for stress reduction",
        "questions": [],
        "findings": [],
        "insights": "",
        "report": "",
        "feedback": "",
        "revision_count": 0,
        "approved": False
    })
    
    print("\n" + "=" * 60)
    print("FINAL REPORT")
    print("=" * 60)
    print(result["report"])


---
### Section 16.5 Exercises

### Exercise 16.5.1: Add a Fact-Checker

Add a fact-checker agent between the researcher and analyst that:
- Reviews each finding for plausibility
- Flags any claims that seem questionable
- Adds confidence levels (high/medium/low) to findings

The analyst should then weight its analysis based on confidence levels.

In [ ]:
# Your code here


### Exercise 16.5.2: Parallel Research

Modify the researcher to gather information from two different "perspectives":
- **Academic perspective** - Focus on research and studies
- **Practical perspective** - Focus on real-world applications

Run both in parallel, then have the analyst synthesize both viewpoints.

In [ ]:
# Your code here


### Exercise 16.5.3: Automatic Gap Detection

Add a "gap detector" agent after the analyst that:
- Reviews the insights and identifies topics that need deeper research
- Automatically triggers additional research on weak areas
- Limits to one round of additional research

The gap detector should check if any of the original questions weren't fully answered and request targeted follow-up research.

In [ ]:
# Your code here


---
## Section 16.6: Managing shared state between agents

In [ ]:
# From: state_patterns.py

# From: Zero to AI Agent, Chapter 16, Section 16.6
# File: state_patterns.py

"""
Demonstrates the three state management patterns in LangGraph:
1. Replace - Agent overwrites a field completely
2. Accumulate - Agent adds to a list
3. Conditional Update - Agent updates only if needed
"""

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
import operator


# =============================================================================
# PATTERN 1: REPLACE
# Agent overwrites a field completely
# =============================================================================

class ReplaceState(TypedDict):
    input_text: str
    processed_text: str  # Will be replaced each time


def processor_replace(state: ReplaceState) -> dict:
    """This REPLACES whatever was in 'processed_text'"""
    result = state["input_text"].upper()
    return {"processed_text": result}


# =============================================================================
# PATTERN 2: ACCUMULATE
# Agent adds to a list using Annotated with operator.add
# =============================================================================

class AccumulateState(TypedDict):
    items_to_process: list[str]
    results: Annotated[list[str], operator.add]  # Will accumulate


def processor_accumulate(state: AccumulateState) -> dict:
    """This APPENDS to the list, doesn't replace"""
    new_results = [item.upper() for item in state["items_to_process"]]
    return {"results": new_results}  # Gets added to existing results


def another_processor(state: AccumulateState) -> dict:
    """This also appends - both results are preserved"""
    new_results = [f"Processed: {item}" for item in state["items_to_process"]]
    return {"results": new_results}


# =============================================================================
# PATTERN 3: CONDITIONAL UPDATE
# Agent updates only if needed
# =============================================================================

class ConditionalState(TypedDict):
    value: int
    threshold: int
    exceeded: bool
    message: str


def conditional_checker(state: ConditionalState) -> dict:
    """Only updates fields when condition is met"""
    if state["value"] > state["threshold"]:
        return {
            "exceeded": True,
            "message": f"Value {state['value']} exceeds threshold {state['threshold']}"
        }
    # Return empty dict if no update needed - fields keep previous values
    return {}


# =============================================================================
# DEMONSTRATION
# =============================================================================

def demo_replace():
    """Demo the replace pattern"""
    print("=" * 50)
    print("PATTERN 1: REPLACE")
    print("=" * 50)
    
    workflow = StateGraph(ReplaceState)
    workflow.add_node("processor", processor_replace)
    workflow.add_edge(START, "processor")
    workflow.add_edge("processor", END)
    app = workflow.compile()
    
    result = app.invoke({
        "input_text": "hello world",
        "processed_text": ""
    })
    
    print(f"Input: 'hello world'")
    print(f"Output: '{result['processed_text']}'")
    print("(Field was replaced with new value)\n")


def demo_accumulate():
    """Demo the accumulate pattern"""
    print("=" * 50)
    print("PATTERN 2: ACCUMULATE")
    print("=" * 50)
    
    workflow = StateGraph(AccumulateState)
    workflow.add_node("proc1", processor_accumulate)
    workflow.add_node("proc2", another_processor)
    workflow.add_edge(START, "proc1")
    workflow.add_edge("proc1", "proc2")
    workflow.add_edge("proc2", END)
    app = workflow.compile()
    
    result = app.invoke({
        "items_to_process": ["a", "b", "c"],
        "results": []
    })
    
    print(f"Input items: ['a', 'b', 'c']")
    print(f"Accumulated results: {result['results']}")
    print("(Both processors' outputs are preserved)\n")


def demo_conditional():
    """Demo the conditional update pattern"""
    print("=" * 50)
    print("PATTERN 3: CONDITIONAL UPDATE")
    print("=" * 50)
    
    workflow = StateGraph(ConditionalState)
    workflow.add_node("checker", conditional_checker)
    workflow.add_edge(START, "checker")
    workflow.add_edge("checker", END)
    app = workflow.compile()
    
    # Test with value below threshold
    result1 = app.invoke({
        "value": 5,
        "threshold": 10,
        "exceeded": False,
        "message": "Initial"
    })
    print(f"Test 1: value=5, threshold=10")
    print(f"  exceeded: {result1['exceeded']}, message: '{result1['message']}'")
    print("  (No update - condition not met)")
    
    # Test with value above threshold
    result2 = app.invoke({
        "value": 15,
        "threshold": 10,
        "exceeded": False,
        "message": "Initial"
    })
    print(f"\nTest 2: value=15, threshold=10")
    print(f"  exceeded: {result2['exceeded']}, message: '{result2['message']}'")
    print("  (Updated - condition met)\n")


if __name__ == "__main__":
    demo_replace()
    demo_accumulate()
    demo_conditional()
    
    print("=" * 50)
    print("KEY TAKEAWAYS:")
    print("=" * 50)
    print("• REPLACE: Default behavior, last write wins")
    print("• ACCUMULATE: Use Annotated[list, operator.add]")
    print("• CONDITIONAL: Return empty dict {} to skip update")


In [ ]:
# From: parallel_state_handling.py

# From: Zero to AI Agent, Chapter 16, Section 16.6
# File: parallel_state_handling.py

"""
Demonstrates how to handle state correctly when agents run in parallel.

Problem: When two agents write to the same field, LangGraph raises an error.
Solutions shown:
1. Separate fields for each agent
2. Accumulate into a shared list
"""

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
import operator


# =============================================================================
# THE PROBLEM: Both agents write to same field
# =============================================================================

class BadState(TypedDict):
    data: str
    result: str  # PROBLEM: Both agents write here!


def agent_a_bad(state: BadState) -> dict:
    return {"result": f"Agent A analyzed: {state['data'][:20]}"}


def agent_b_bad(state: BadState) -> dict:
    return {"result": f"Agent B analyzed: {state['data'][:20]}"}  # Conflict!


def demo_problem():
    """Show the problem with parallel agents writing to same field"""
    print("=" * 60)
    print("PROBLEM: Parallel agents writing to same field")
    print("=" * 60)
    
    workflow = StateGraph(BadState)
    workflow.add_node("agent_a", agent_a_bad)
    workflow.add_node("agent_b", agent_b_bad)
    
    # Both run in parallel from START
    workflow.add_edge(START, "agent_a")
    workflow.add_edge(START, "agent_b")
    workflow.add_edge("agent_a", END)
    workflow.add_edge("agent_b", END)
    
    app = workflow.compile()
    
    try:
        result = app.invoke({
            "data": "Some text to analyze",
            "result": ""
        })
        # Old behavior (silent overwrite)
        print(f"Result: '{result['result']}'")
        print("⚠️  Only ONE agent's result is preserved!")
        print("   The other was overwritten.\n")
    except Exception as e:
        # New behavior (LangGraph catches the conflict)
        print(f"\n❌ LangGraph caught the conflict!")
        print(f"   Error: {type(e).__name__}")
        print(f"   Message: Can receive only one value per step.")
        print("\n✅ This is actually GOOD - LangGraph prevents data loss!")
        print("   Let's see how to fix it properly...\n")


# =============================================================================
# SOLUTION 1: Separate fields for each agent
# =============================================================================

class SeparateFieldsState(TypedDict):
    data: str
    result_a: str  # Agent A owns this
    result_b: str  # Agent B owns this
    combined: str  # Merger owns this


def agent_a_separate(state: SeparateFieldsState) -> dict:
    return {"result_a": f"Agent A: {state['data'][:20]}..."}


def agent_b_separate(state: SeparateFieldsState) -> dict:
    return {"result_b": f"Agent B: {state['data'][:20]}..."}


def merger_separate(state: SeparateFieldsState) -> dict:
    return {"combined": f"{state['result_a']}\n{state['result_b']}"}


def demo_solution_1():
    """Demo solution with separate fields"""
    print("=" * 60)
    print("SOLUTION 1: Separate fields for each agent")
    print("=" * 60)
    
    workflow = StateGraph(SeparateFieldsState)
    workflow.add_node("agent_a", agent_a_separate)
    workflow.add_node("agent_b", agent_b_separate)
    workflow.add_node("merger", merger_separate)
    
    # Parallel execution
    workflow.add_edge(START, "agent_a")
    workflow.add_edge(START, "agent_b")
    
    # Both feed into merger
    workflow.add_edge("agent_a", "merger")
    workflow.add_edge("agent_b", "merger")
    
    workflow.add_edge("merger", END)
    
    app = workflow.compile()
    
    result = app.invoke({
        "data": "Some text to analyze for demonstration",
        "result_a": "",
        "result_b": "",
        "combined": ""
    })
    
    print(f"Agent A result: '{result['result_a']}'")
    print(f"Agent B result: '{result['result_b']}'")
    print(f"Combined: '{result['combined']}'")
    print("✅ Both results preserved!\n")


# =============================================================================
# SOLUTION 2: Accumulate into a shared list
# =============================================================================

class AccumulateState(TypedDict):
    data: str
    results: Annotated[list[str], operator.add]  # Both agents add to this


def agent_a_accumulate(state: AccumulateState) -> dict:
    return {"results": [f"Agent A: {state['data'][:20]}..."]}


def agent_b_accumulate(state: AccumulateState) -> dict:
    return {"results": [f"Agent B: {state['data'][:20]}..."]}


def demo_solution_2():
    """Demo solution with accumulation"""
    print("=" * 60)
    print("SOLUTION 2: Accumulate into a shared list")
    print("=" * 60)
    
    workflow = StateGraph(AccumulateState)
    workflow.add_node("agent_a", agent_a_accumulate)
    workflow.add_node("agent_b", agent_b_accumulate)
    
    # Parallel execution
    workflow.add_edge(START, "agent_a")
    workflow.add_edge(START, "agent_b")
    workflow.add_edge("agent_a", END)
    workflow.add_edge("agent_b", END)
    
    app = workflow.compile()
    
    result = app.invoke({
        "data": "Some text to analyze for demonstration",
        "results": []
    })
    
    print(f"Accumulated results: {result['results']}")
    print("✅ Both results preserved in list!\n")


# =============================================================================
# BEST PRACTICE: Clear field ownership
# =============================================================================

def show_ownership_pattern():
    """Show the ownership pattern we use"""
    print("=" * 60)
    print("BEST PRACTICE: Clear field ownership")
    print("=" * 60)
    print("""
In our research team (Section 16.5):

    Agent          | Owns Field      | Reads From
    ---------------|-----------------|------------------
    Planner        | questions       | topic
    Researcher     | findings        | questions
    Analyst        | insights        | findings
    Writer         | report          | insights, findings
    Reviewer       | approved, feedback | report

No conflicts because each agent writes to different fields!
""")


if __name__ == "__main__":
    demo_problem()
    demo_solution_1()
    demo_solution_2()
    show_ownership_pattern()

---
### Section 16.6 Exercises

### Exercise 16.6.1: State Audit

Review the research team from Section 16.5. For each field in `ResearchState`:
1. Which agent writes to it?
2. Which agents read from it?
3. Is it replaced or accumulated?

Create a simple diagram showing the data flow.

In [ ]:
# Your code here


### Exercise 16.6.2: Fix the Bug

This state has a problem when used with parallel agents. Identify and fix it:

```python
class AnalysisState(TypedDict):
    data: str
    result: str  # Both analyzers write here!

def technical_analyzer(state) -> dict:
    return {"result": f"Technical: {analyze(state['data'])}"}

def business_analyzer(state) -> dict:
    return {"result": f"Business: {analyze(state['data'])}"}
```

In [ ]:
# Your code here


### Exercise 16.6.3: Design a State

Design a state schema for a content moderation system with these agents:
- **Toxicity Checker** - Scores content for toxic language
- **Spam Detector** - Checks if content is spam
- **PII Scanner** - Finds personal information
- **Decision Maker** - Makes final allow/block decision

Consider: What fields does each agent need? How should scores be stored? What does the decision maker need to see?

In [ ]:
# Your code here


---
## Section 16.7: Orchestration and coordination

In [ ]:
# From: orchestrated_system.py

# From: Zero to AI Agent, Chapter 16, Section 16.7
# File: orchestrated_system.py

"""
Template for a well-orchestrated multi-agent system.

Features:
- Logging for observability
- Error handling with retries
- Fallback for graceful degradation
- Clean state management
- Clear routing logic
"""

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from datetime import datetime
from dotenv import load_dotenv
import logging

load_dotenv()

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("orchestrator")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)


# =============================================================================
# STATE
# =============================================================================

class OrchestratedState(TypedDict):
    task: str
    stage: str
    result: str
    error: str
    attempts: int
    max_attempts: int


# =============================================================================
# AGENTS WITH ERROR HANDLING
# =============================================================================

def worker_agent(state: OrchestratedState) -> dict:
    """Worker with built-in error handling."""
    logger.info(f"Worker starting (attempt {state.get('attempts', 0) + 1})")
    
    try:
        response = llm.invoke(f"Complete this task briefly: {state['task']}")
        logger.info("Worker completed successfully")
        return {
            "result": response.content,
            "stage": "complete",
            "error": ""
        }
    except Exception as e:
        logger.error(f"Worker failed: {e}")
        return {
            "error": str(e),
            "attempts": state.get("attempts", 0) + 1
        }


def fallback_agent(state: OrchestratedState) -> dict:
    """Fallback when primary worker fails."""
    logger.warning("Using fallback agent")
    return {
        "result": f"Unable to fully complete: {state['task']}. Please try again.",
        "stage": "fallback"
    }


# =============================================================================
# ROUTING
# =============================================================================

def check_result(state: OrchestratedState) -> Literal["done", "retry", "fallback"]:
    """Decide next step based on result."""
    if state.get("result") and not state.get("error"):
        return "done"
    
    attempts = state.get("attempts", 0)
    max_attempts = state.get("max_attempts", 3)
    
    if attempts >= max_attempts:
        return "fallback"
    
    return "retry"


# =============================================================================
# BUILD GRAPH
# =============================================================================

workflow = StateGraph(OrchestratedState)

workflow.add_node("worker", worker_agent)
workflow.add_node("fallback", fallback_agent)

workflow.add_edge(START, "worker")

workflow.add_conditional_edges(
    "worker",
    check_result,
    {
        "done": END,
        "retry": "worker",
        "fallback": "fallback"
    }
)

workflow.add_edge("fallback", END)

app = workflow.compile()


# =============================================================================
# RUN
# =============================================================================

if __name__ == "__main__":
    result = app.invoke({
        "task": "Explain what an API is in one sentence",
        "stage": "starting",
        "result": "",
        "error": "",
        "attempts": 0,
        "max_attempts": 3
    })
    
    print(f"\nResult: {result['result']}")
    print(f"Stage: {result['stage']}")


In [ ]:
# From: content_creation_pipeline.py

# From: Zero to AI Agent, Chapter 16 Challenge Project
# File: content_creation_pipeline.py

"""
Chapter 16 Challenge Project: Content Creation Pipeline

Multi-agent system that produces blog posts with:
- Topic Researcher: Gathers facts and background
- Outline Creator: Structures content into sections
- Draft Writer: Writes initial draft
- Editor: Reviews and improves draft
- SEO Optimizer: Adds keywords and improves searchability

Features:
- Pipeline flow: Research → Outline → Draft → Edit → SEO
- Quality loop: Editor can send back to Writer (max 2 revisions)
- Error handling with fallbacks
- Metrics reporting
"""

from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from datetime import datetime
import operator
import logging

load_dotenv()

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("content_pipeline")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


# =============================================================================
# STATE
# =============================================================================

class ContentState(TypedDict):
    topic: str
    research: str
    outline: str
    draft: str
    edited_draft: str
    final_content: str
    feedback: str
    revision_count: int
    max_revisions: int
    approved: bool
    stage_times: Annotated[list[str], operator.add]  # Track time per stage


# =============================================================================
# HELPER: Time tracker
# =============================================================================

def track_time(agent_name: str, start: datetime) -> str:
    """Create a time tracking entry."""
    duration = (datetime.now() - start).total_seconds()
    return f"{agent_name}: {duration:.2f}s"


# =============================================================================
# AGENTS
# =============================================================================

def topic_researcher(state: ContentState) -> dict:
    """Gathers facts and background on the topic."""
    start = datetime.now()
    logger.info(f"[Researcher] Starting research on: {state['topic']}")
    
    try:
        prompt = f"""Research the topic: {state['topic']}

Provide:
1. Key facts (3-4 bullet points)
2. Background context
3. Current relevance/trends
4. Target audience interests

Keep it concise but informative."""

        response = llm.invoke(prompt)
        
        logger.info("[Researcher] Research complete")
        return {
            "research": response.content,
            "stage_times": [track_time("Researcher", start)]
        }
    except Exception as e:
        logger.error(f"[Researcher] Error: {e}")
        return {
            "research": f"Basic research on {state['topic']}: A popular and relevant topic.",
            "stage_times": [track_time("Researcher (fallback)", start)]
        }


def outline_creator(state: ContentState) -> dict:
    """Creates a structured outline for the content."""
    start = datetime.now()
    logger.info("[Outline] Creating content structure")
    
    try:
        prompt = f"""Based on this research, create a blog post outline:

TOPIC: {state['topic']}

RESEARCH:
{state['research']}

Create an outline with:
- Catchy title
- Introduction hook
- 3-4 main sections with subpoints
- Conclusion with call-to-action

Format as a clear outline structure."""

        response = llm.invoke(prompt)
        
        logger.info("[Outline] Structure created")
        return {
            "outline": response.content,
            "stage_times": [track_time("Outline Creator", start)]
        }
    except Exception as e:
        logger.error(f"[Outline] Error: {e}")
        return {
            "outline": f"I. Introduction\nII. Main Points about {state['topic']}\nIII. Conclusion",
            "stage_times": [track_time("Outline Creator (fallback)", start)]
        }


def draft_writer(state: ContentState) -> dict:
    """Writes the initial draft based on outline."""
    start = datetime.now()
    revision_note = f" (revision {state['revision_count']})" if state['revision_count'] > 0 else ""
    logger.info(f"[Writer] Writing draft{revision_note}")
    
    try:
        if state['revision_count'] > 0 and state['feedback']:
            # Revision mode - incorporate feedback
            prompt = f"""Revise this blog post based on the feedback:

CURRENT DRAFT:
{state['edited_draft'] or state['draft']}

EDITOR FEEDBACK:
{state['feedback']}

Write an improved version that addresses all feedback points.
Maintain the overall structure but improve quality."""
        else:
            # Initial draft
            prompt = f"""Write a blog post based on this outline:

TOPIC: {state['topic']}

OUTLINE:
{state['outline']}

RESEARCH:
{state['research']}

Write engaging, informative content:
- Use conversational tone
- Include specific facts from research
- Make it 300-400 words
- Use subheadings for sections"""

        response = llm.invoke(prompt)
        
        logger.info("[Writer] Draft complete")
        return {
            "draft": response.content,
            "stage_times": [track_time(f"Writer{revision_note}", start)]
        }
    except Exception as e:
        logger.error(f"[Writer] Error: {e}")
        return {
            "draft": f"# {state['topic']}\n\nContent about {state['topic']}...",
            "stage_times": [track_time("Writer (fallback)", start)]
        }


def editor(state: ContentState) -> dict:
    """Reviews and provides feedback on the draft."""
    start = datetime.now()
    logger.info(f"[Editor] Reviewing draft (revision {state['revision_count']})")
    
    try:
        prompt = f"""Review this blog post draft:

{state['draft']}

Evaluate:
1. Content quality (facts, depth)
2. Writing clarity
3. Engagement level
4. Structure and flow
5. Length (should be 300+ words)

If the draft is good (score 7+/10), respond with:
APPROVED: Yes
FEEDBACK: Brief praise

If it needs work, respond with:
APPROVED: No
FEEDBACK: Specific improvements needed (bullet points)"""

        response = llm.invoke(prompt)
        result = response.content
        
        # Parse response
        approved = "APPROVED: Yes" in result or "approved: yes" in result.lower()
        
        # Extract feedback
        if "FEEDBACK:" in result:
            feedback = result.split("FEEDBACK:")[-1].strip()
        else:
            feedback = result
        
        logger.info(f"[Editor] Review complete - Approved: {approved}")
        return {
            "edited_draft": state['draft'],
            "feedback": feedback,
            "approved": approved,
            "revision_count": state['revision_count'] + 1,
            "stage_times": [track_time("Editor", start)]
        }
    except Exception as e:
        logger.error(f"[Editor] Error: {e}")
        return {
            "edited_draft": state['draft'],
            "approved": True,  # Approve on error to continue
            "feedback": "Auto-approved due to processing issue",
            "revision_count": state['revision_count'] + 1,
            "stage_times": [track_time("Editor (fallback)", start)]
        }


def seo_optimizer(state: ContentState) -> dict:
    """Adds SEO keywords and improves searchability."""
    start = datetime.now()
    logger.info("[SEO] Optimizing for search")
    
    try:
        prompt = f"""Optimize this blog post for SEO:

{state['edited_draft'] or state['draft']}

Add:
1. SEO-friendly title (with main keyword)
2. Meta description (150 chars)
3. 3-5 relevant keywords naturally in text
4. Internal/external link suggestions
5. Alt text suggestions for potential images

Return the optimized full article with SEO elements clearly marked."""

        response = llm.invoke(prompt)
        
        logger.info("[SEO] Optimization complete")
        return {
            "final_content": response.content,
            "stage_times": [track_time("SEO Optimizer", start)]
        }
    except Exception as e:
        logger.error(f"[SEO] Error: {e}")
        return {
            "final_content": state['edited_draft'] or state['draft'],
            "stage_times": [track_time("SEO Optimizer (fallback)", start)]
        }


# =============================================================================
# ROUTING
# =============================================================================

def should_revise(state: ContentState) -> Literal["revise", "optimize"]:
    """Decide if draft needs revision or can proceed to SEO."""
    if state['approved']:
        logger.info("[Router] Draft approved -> SEO")
        return "optimize"
    
    if state['revision_count'] >= state['max_revisions']:
        logger.info(f"[Router] Max revisions ({state['max_revisions']}) reached -> SEO")
        return "optimize"
    
    logger.info(f"[Router] Revision {state['revision_count']} requested -> Writer")
    return "revise"


# =============================================================================
# BUILD WORKFLOW
# =============================================================================

workflow = StateGraph(ContentState)

# Add nodes
workflow.add_node("researcher", topic_researcher)
workflow.add_node("outline", outline_creator)
workflow.add_node("writer", draft_writer)
workflow.add_node("editor", editor)
workflow.add_node("seo", seo_optimizer)

# Add edges
workflow.add_edge(START, "researcher")
workflow.add_edge("researcher", "outline")
workflow.add_edge("outline", "writer")
workflow.add_edge("writer", "editor")

# Conditional edge for revision loop
workflow.add_conditional_edges(
    "editor",
    should_revise,
    {
        "revise": "writer",
        "optimize": "seo"
    }
)

workflow.add_edge("seo", END)

content_pipeline = workflow.compile()


# =============================================================================
# METRICS REPORT
# =============================================================================

def print_metrics(state: ContentState):
    """Print stage timing metrics."""
    print("\n" + "=" * 50)
    print("PIPELINE METRICS")
    print("=" * 50)
    
    total_time = 0
    for entry in state['stage_times']:
        print(f"  {entry}")
        # Extract time from entry like "Writer: 1.23s"
        try:
            time_str = entry.split(":")[-1].strip().replace("s", "")
            total_time += float(time_str)
        except:
            pass
    
    print("-" * 50)
    print(f"  Total: {total_time:.2f}s")
    print(f"  Revisions: {state['revision_count']}")
    print(f"  Approved: {state['approved']}")
    print("=" * 50)


# =============================================================================
# RUN
# =============================================================================

if __name__ == "__main__":
    print("=" * 60)
    print("CONTENT CREATION PIPELINE")
    print("=" * 60)
    
    topic = "The Benefits of Learning Python in 2024"
    print(f"\nTopic: {topic}\n")
    
    result = content_pipeline.invoke({
        "topic": topic,
        "research": "",
        "outline": "",
        "draft": "",
        "edited_draft": "",
        "final_content": "",
        "feedback": "",
        "revision_count": 0,
        "max_revisions": 2,
        "approved": False,
        "stage_times": []
    })
    
    # Print metrics
    print_metrics(result)
    
    # Print final content
    print("\n" + "=" * 60)
    print("FINAL OPTIMIZED CONTENT")
    print("=" * 60)
    print(result['final_content'])


---
### Section 16.7 Exercises

### Exercise 16.7.1: Add Monitoring

Take the research team from Section 16.5 and add:
- Logging for each agent (start/complete/duration)
- A simple metrics report at the end
- Error counting per agent

In [ ]:
# Your code here


### Exercise 16.7.2: Implement Timeout

Create a wrapper that:
- Gives each agent a maximum time to complete
- Returns a default response if timeout is exceeded
- Logs timeout events

In [ ]:
# Your code here


### Exercise 16.7.3: Build a Health Check

Create a health check system that:
- Tests each agent with a simple input
- Reports which agents are working
- Flags agents with high error rates

In [ ]:
# Your code here


---
## Next Steps

- Check your answers in **chapter_16_multi_agent_solutions.ipynb**
- Proceed to **Chapter 17**